<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Unsupervised_Similar_Images_Finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**I**n this project we are going to learn about how to fins similar images of and select the best one based on it's size(resolution) and store it in the main folder and send the other copies to the rubish folder

In [3]:
import os
import cv2
import glob
import tqdm
import shutil
import skimage
import IPython
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

!pip install patool
import patoolib

IPython.display.clear_output()

Step 1: Read the zip file from the drive and unzip it

In [4]:
patoolib.extract_archive("/content/drive/MyDrive/reddit_files.rar", outdir="/content/data")

Path = './data/images/'
os.rename(f"./data/{os.listdir('./data')[0]}", Path)

IPython.display.clear_output()

data = os.listdir(Path)
len(data)

2009

Step 2: create a path for the videos and rubbish data and store the video clips in the video folder and then select the images andsave it as data variable

In [ ]:
Path = './data/images/'
rubish_path = './Rubish/'
data = glob.glob(f'{Path}/*g')
videos = glob.glob(f'{Path}/*.mp4')
os.makedirs(f'{Path}/Videos/', exist_ok=True)
os.makedirs(rubish_path, exist_ok=True)
for i, video_path in enumerate(videos):
    os.replace(video_path, f'{Path}/Videos/{i}.mp4')

Step 3:
create a Mean-Pooling layer

Reason:
an image is a big matrix and since we only need to compare the images from some of their pixels, we need to at first convert 3d image to a 1-dim image and then get the mean part of each block of the image

In [ ]:
def avg_pooling(img, pad_size=20):
  image = abs(img[:, :, 0] - img[:, :, 1] - img[:, :, 2])
  max_pool_img = skimage.measure.block_reduce(image, (pad_size,pad_size), np.mean)
  return max_pool_img

Step 4:
we have to read the images and since we are using min-pooling, we have to resize them in a same size, for example:(1000, 1000) and then pass it through avg-pooling then flatten the image, so we have less values per image

In [ ]:
arr = []
height_arr, weight_arr = [], []
for d in tqdm.tqdm(data):
  try:
      img = cv2.imread(d)
      height_arr.append(img.shape[0])
      weight_arr.append(img.shape[1])
      img = cv2.resize(img, (1000, 1000))
      img = avg_pooling(img, pad_size=50)
      img = np.ravel(img)
      arr.append(img)
  except:
    print(d)

print('\n')
print('Height Mean:',np.mean(height_arr), 'Height Min:',np.min(height_arr), 'Height Max:',np.max(height_arr))
print('Weight Mean:',np.mean(weight_arr), 'Weight Min:',np.min(weight_arr), 'Weight Max:',np.max(weight_arr))

100%|██████████| 1970/1970 [01:29<00:00, 22.00it/s]



Height Mean: 1343.7598984771573 Height Min: 194 Height Max: 8598
Weight Mean: 1111.6837563451777 Weight Min: 149 Weight Max: 6144


Step 5:
In this step we go through the images that we stored in arr and get the average of the difference of each image values from every other images in the directory, and only select the ones that have mean average error < 15

at last we sort the selected images in temp_data and sort it by temp_size because we want the images with higher resolution(bigger sizes) and store the sorted image pathes in to mae_err

In [ ]:
mae_arr = []
for i, A in tqdm.tqdm(enumerate(arr)):
  temp_data = []
  temp_size = []
  mean_mae_arr = [(abs(A - b)).mean() for b in arr]
  for j, mae in enumerate(mean_mae_arr):
    if mae < 15:
      img = cv2.imread(data[j])
      img_size = img.shape[0] * img.shape[1]
      temp_data.append(data[j])
      temp_size.append(img_size)

  temp_data = np.array(temp_data)[np.argsort(temp_size)[::-1]]
  mae_arr.append(temp_data)

mae_arr = np.array(mae_arr)
mae_arr.shape

Step 6:
In the last step, we store the first images of each path_list and get rid of the othe copies

In [ ]:
for id, path_list in tqdm.tqdm(enumerate(mae_arr)):
  for n, p in enumerate(path_list):
    if os.path.exists(p):
      if n==0:
        os.replace(p, f'{Path}/img_{id}.jpg')
      else:
        os.replace(p, f'{rubish_path}/img_{id}_{n}.jpg')

1970it [00:00, 18019.87it/s]


Functional model

In [9]:
def similar_finder(Path, rubish_path):
  #Part 1
  data = glob.glob(f'{Path}/*g')
  videos = glob.glob(f'{Path}/*.mp4')
  os.makedirs(f'{Path}/Videos/', exist_ok=True)
  os.makedirs(rubish_path, exist_ok=True)
  for i, video_path in enumerate(videos):
      os.replace(video_path, f'{Path}/Videos/{i}.mp4')

  #Part 2
  img_arr = []
  for d in tqdm.tqdm(data):
    try:
        img = cv2.imread(d)
        img = cv2.resize(img, (1000, 1000))
        #Mean Average Pooling
        img = skimage.measure.block_reduce(abs(img[:, :, 0] - img[:, :, 1] - img[:, :, 2]), (50, 50), np.mean)
        img = np.ravel(img)
        img_arr.append(img)
    except:
      print('Error File: ', d)

  #Part 3
  mae_arr = []
  for i, A in tqdm.tqdm(enumerate(img_arr)):
    temp_path = []
    temp_size = []
    mean_mae_arr = [(abs(A - b)).mean() for b in img_arr]
    for j, mae in enumerate(mean_mae_arr):
      if mae < 15:
        img = cv2.imread(data[j])
        img_size = img.shape[0] * img.shape[1]
        temp_path.append(data[j])
        temp_size.append(img_size)

    temp_path = np.array(temp_path)[np.argsort(temp_size)[::-1]]
    mae_arr.append(temp_path)

  #Part 4
  for id, path_list in tqdm.tqdm(enumerate(mae_arr)):
    for n, p in enumerate(path_list):

      if os.path.exists(p):
        if n==0:
          os.replace(p, f'{Path}/img_{id}.jpg')
        else:
          os.replace(p, f'{rubish_path}/img_{id}_{n}.jpg')

In [10]:
Path = './data/images/'
rubish_path = './Rubish/'

similar_finder(Path = './data/images/', rubish_path = './Rubish/')

100%|██████████| 1970/1970 [01:18<00:00, 25.10it/s]
1970it [02:54, 11.31it/s]
1970it [00:00, 19511.43it/s]
